In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import *
from pyspark.sql.types import *

In [0]:
df_athletes=spark.read.format("csv").option("header",True).option("delimiter",",").option("inferscheam",True).load("/FileStore/tables/Olympic/Athletes.csv")
df_athletes.show(5,truncate=False)

+-----------------+-------+-------------------+
|PersonName       |Country|Discipline         |
+-----------------+-------+-------------------+
|AALERUD Katrine  |Norway |Cycling Road       |
|ABAD Nestor      |Spain  |Artistic Gymnastics|
|ABAGNALE Giovanni|Italy  |Rowing             |
|ABALDE Alberto   |Spain  |Basketball         |
|ABALDE Tamara    |Spain  |Basketball         |
+-----------------+-------+-------------------+
only showing top 5 rows



In [0]:
df_coaches=spark.read.format("csv").option("header",True).option("delimiter",",").option("inferscheam",True).load("/FileStore/tables/Olympic/Coaches.csv")
df_coaches.show(5,truncate=False)

+---------------+-------------+----------+-----+
|Name           |Country      |Discipline|Event|
+---------------+-------------+----------+-----+
|ABDELMAGID Wael|Egypt        |Football  |null |
|ABE Junya      |Japan        |Volleyball|null |
|ABE Katsuhiko  |Japan        |Basketball|null |
|ADAMA Cherif   |C�te d'Ivoire|Football  |null |
|AGEBA Yuya     |Japan        |Volleyball|null |
+---------------+-------------+----------+-----+
only showing top 5 rows



In [0]:
# Identifing the Coaches who trained the Athlete
df_athletes=df_athletes.alias("athletes")
df_coaches=df_coaches.alias("coaches")
df=df_athletes.join(df_coaches,(df_athletes.Discipline==df_coaches.Discipline)&(df_athletes.Country==df_coaches.Country),"Left")\
    .select("athletes.PersonName","coaches.Name","coaches.Discipline").filter(df_coaches.Name.isNotNull()).distinct()
df.show(truncate=False)

+--------------+-----------------------------+----------+
|PersonName    |Name                         |Discipline|
+--------------+-----------------------------+----------+
|ABALDE Tamara |SCARIOLO Sergio              |Basketball|
|ABALDE Tamara |LAZARO CORRAL Jesus          |Basketball|
|ABALDE Tamara |GONZALEZ JARENO Angel        |Basketball|
|ABALDE Tamara |MIRET GARCIA Daniel          |Basketball|
|ABALDE Alberto|DIAZ FERNANDEZ Jenaro Manuel |Basketball|
|ABALDE Alberto|ALLER CARBALLO Manuel Angel  |Basketball|
|ABALDE Alberto|MIRET GARCIA Daniel          |Basketball|
|ABALDE Tamara |ALLER CARBALLO Manuel Angel  |Basketball|
|ABALDE Tamara |ZAMORA PEDREIRA Javier       |Basketball|
|ABALDE Alberto|ZAMORA PEDREIRA Javier       |Basketball|
|ABALDE Tamara |HERNANDEZ FRAILE Jose Ignacio|Basketball|
|ABALO Luc     |KRUMBHOLZ Olivier            |Handball  |
|ABALDE Alberto|MONDELO GARCIA Lucas         |Basketball|
|ABALDE Alberto|HERNANDEZ FRAILE Jose Ignacio|Basketball|
|ABALDE Tamara

In [0]:
# Display total sports played by each country and rank them based on the games played
df=df_athletes.select("Country","Discipline").groupBy(col("Country")).agg(count("Discipline").alias("TotalSportsPlayed"))
df=df.withColumn("ranking",dense_rank().over(Window.orderBy(col("TotalSportsPlayed").asc())))
df.show(truncate=False)

+-----------------------------+-----------------+-------+
|Country                      |TotalSportsPlayed|ranking|
+-----------------------------+-----------------+-------+
|Dominica                     |2                |1      |
|Andorra                      |2                |1      |
|Lesotho                      |2                |1      |
|Myanmar                      |2                |1      |
|Brunei Darussalam            |2                |1      |
|Somalia                      |2                |1      |
|Marshall Islands             |2                |1      |
|Tuvalu                       |2                |1      |
|United Republic of Tanzania  |2                |1      |
|St Vincent and the Grenadines|2                |1      |
|Mauritania                   |2                |1      |
|Central African Republic     |2                |1      |
|Nauru                        |2                |1      |
|Saint Kitts and Nevis        |2                |1      |
|South Sudan  

In [0]:
# Display the country which have played Volleyball and also the count of player palying for each country
df=df_athletes.select("Country","PersonName").groupBy("Country").agg(count("PersonName").alias("number_of_player_played")).drop("PersonName")
df.show(truncate=False)

+------------------------+-----------------------+
|Country                 |number_of_player_played|
+------------------------+-----------------------+
|Chad                    |3                      |
|Paraguay                |8                      |
|Yemen                   |3                      |
|Islamic Republic of Iran|66                     |
|Chinese Taipei          |67                     |
|Senegal                 |9                      |
|Sweden                  |129                    |
|Kiribati                |3                      |
|Republic of Korea       |223                    |
|Guyana                  |7                      |
|Philippines             |18                     |
|Eritrea                 |13                     |
|Djibouti                |4                      |
|Tonga                   |5                      |
|Malaysia                |29                     |
|Singapore               |23                     |
|Fiji                    |28   

In [0]:
#  Using the above dataframe display which country has the 5th rank in amount of players played
df=df.withColumn("ranking",dense_rank().over(Window.orderBy("number_of_player_played")))
df=df.filter(df.ranking==5)
df.show(truncate=False)

+--------------------+-----------------------+-------+
|Country             |number_of_player_played|ranking|
+--------------------+-----------------------+-------+
|Madagascar          |6                      |5      |
|Malta               |6                      |5      |
|Burundi             |6                      |5      |
|Bangladesh          |6                      |5      |
|Monaco              |6                      |5      |
|Cape Verde          |6                      |5      |
|Grenada             |6                      |5      |
|Cook Islands        |6                      |5      |
|Lebanon             |6                      |5      |
|Antigua and Barbuda |6                      |5      |
|Haiti               |6                      |5      |
|Syrian Arab Republic|6                      |5      |
+--------------------+-----------------------+-------+



In [0]:
# Display the ranks of the country based on silver gold and bronze individually and print 7th rank for each catogory
schema=StructType([StructField("Rank",IntegerType()),\
    StructField("Team_Country",StringType()),\
    StructField("Gold",IntegerType()),\
    StructField("Silver",IntegerType()),\
    StructField("Bronze",IntegerType()),\
    StructField("Total",IntegerType()),\
    StructField("Rank_by_Total",IntegerType())])
df_medals=spark.read.format("csv").option("header",True).option("delimiter",",").schema(schema).load("/FileStore/tables/Olympic/Medals.csv")
    
# Creating the Rank based on the medal counts
df=df_medals.withColumn("Gold_ranking",dense_rank().over(Window.orderBy(col("Gold").desc())))\
    .withColumn("Silver_ranking",dense_rank().over(Window.orderBy(col("Silver").desc())))\
    .withColumn("Bronze_ranking",dense_rank().over(Window.orderBy(col("Bronze").desc())))
df_gold=df.select("Team_Country","Gold_ranking").withColumnRenamed("Gold_ranking","Medal_ranking").withColumn("Medal_type",lit("Gold")).filter(col("Gold_ranking")==7)
df_silver=df.select("Team_Country","Silver_ranking").withColumnRenamed("Silver_ranking","Medal_ranking").withColumn("Medal_type",lit("Silver")).filter(col("Silver_ranking")==7)
df_bronze=df.select("Team_Country","Bronze_ranking").withColumnRenamed("Bronze_ranking","Medal_ranking").withColumn("Medal_type",lit("Bronze")).filter(col("Bronze_ranking")==7)
final_df=df_gold.union(df_silver)
final_df=final_df.union(df_bronze)
display(final_df)

Team_Country,Medal_ranking,Medal_type
Netherlands,7,Gold
France,7,Gold
Germany,7,Gold
Italy,7,Gold
Germany,7,Silver
Germany,7,Bronze


In [0]:
df=df_medals.withColumn("Gold_ranking",dense_rank().over(Window.orderBy(col("Gold").desc())))\
    .withColumn("Silver_ranking",dense_rank().over(Window.orderBy(col("Silver").desc())))\
    .withColumn("Bronze_ranking",dense_rank().over(Window.orderBy(col("Bronze").desc())))
df_gold=df.select("Team_Country","Gold_ranking").filter(col("Gold_ranking")==7)
df_silver=df.select("Team_Country","Silver_ranking").filter(col("Silver_ranking")==7)
df_bronze=df.select("Team_Country","Bronze_ranking").filter(col("Bronze_ranking")==7)

final_df=df_gold.unionByName(df_silver,True)
final_df=final_df.unionByName(df_bronze,True)
display(final_df)

Team_Country,Gold_ranking,Silver_ranking,Bronze_ranking
Netherlands,7,null,null
France,7,null,null
Germany,7,null,null
Italy,7,null,null
Germany,null,7,null
Germany,null,null,7


In [0]:
# Display the total count of teams played played for each country in women event
df_teams=spark.read.format("csv").option("header",True).option("delimiter",",").option("inferscheam",True).load("/FileStore/tables/Olympic/Teams.csv")

df=df_teams.select("TeamName","Event").filter(col("Event").contains("Women")).groupBy("TeamName").agg(count(col("event")).alias("Women_teams_played"))
df.show(truncate=False)

+---------------------+------------------+
|TeamName             |Women_teams_played|
+---------------------+------------------+
|Graudina/Kravcenoka  |1                 |
|Pavan/Melissa        |1                 |
|Huberli/Betschart    |1                 |
|Keizer/Meppelink     |1                 |
|Chinese Taipei       |2                 |
|Sweden               |4                 |
|Wang/X.Y.Xia         |1                 |
|Republic of Korea    |8                 |
|Makroguzova/Kholomina|1                 |
|Singapore            |1                 |
|Fiji                 |1                 |
|Turkey               |2                 |
|Germany              |12                |
|France               |14                |
|Hong Kong, China     |6                 |
|Argentina            |2                 |
|Belgium              |4                 |
|Angola               |1                 |
|Great Britain        |11                |
|Ecuador              |1                 |
+----------

In [0]:
# Display the country which have 0 Bronze and have both silver and gold medal and rank them based on total count of medals
df=df_medals.select("Team_Country","Gold","Silver","Bronze").filter((col("Bronze")==0)&((col("Gold")!=0)&((col("Silver")!=0))))
df=df.withColumn("Total_medals",col("Gold")+col("Silver"))
df=df.withColumn("Ranking",dense_rank().over(Window.orderBy(col("Total_medals").desc()))).drop("Bronze")
df.show(truncate=False)

+------------+----+------+------------+-------+
|Team_Country|Gold|Silver|Total_medals|Ranking|
+------------+----+------+------------+-------+
|Sweden      |3   |6     |9           |1      |
|Romania     |1   |3     |4           |2      |
|Venezuela   |1   |3     |4           |2      |
|Ecuador     |2   |1     |3           |3      |
|South Africa|1   |2     |3           |3      |
|Tunisia     |1   |1     |2           |4      |
+------------+----+------+------------+-------+



In [0]:
# Display the rolling sum of total medals won in all the 3 position(silver,gold,bronze)
df=df_medals.withColumn("Rolling_Gold",sum("Gold").over(Window.orderBy("Gold")))\
    .withColumn("Rolling_Silver",sum("Silver").over(Window.orderBy("Silver")))\
    .withColumn("Rolling_Bronze",sum("Bronze").over(Window.orderBy("Bronze"))).drop("Gold","Silver","Bronze","Rank","Total","Rank_by_Total")
df.show(truncate=False)

+---------------+------------+--------------+--------------+
|Team_Country   |Rolling_Gold|Rolling_Silver|Rolling_Bronze|
+---------------+------------+--------------+--------------+
|Bermuda        |22          |0             |0             |
|Morocco        |22          |0             |0             |
|Puerto Rico    |22          |0             |0             |
|Bahamas        |44          |0             |0             |
|Kosovo         |44          |0             |0             |
|Bahrain        |0           |26            |0             |
|Saudi Arabia   |0           |26            |0             |
|Lithuania      |0           |26            |0             |
|North Macedonia|0           |26            |0             |
|Namibia        |0           |26            |0             |
|Turkmenistan   |0           |26            |0             |
|Tunisia        |22          |26            |0             |
|Ecuador        |44          |26            |0             |
|South Africa   |22     

In [0]:
# Display all the Sports where women players are more in comaired to men players
schema=StructType([StructField("Discipline",StringType(),True),\
    StructField("Female",IntegerType(),True),\
    StructField("Male",IntegerType(),True),\
    StructField("Total",IntegerType(),True)])
df_entries_gender=spark.read.format("csv").option("header",True).option("delimiter",",").schema(schema).load("/FileStore/tables/Olympic/EntriesGender.csv")


df=df_entries_gender.select("Discipline","Female","Male").filter(col("Female")>col("Male"))
df.show(truncate=False)

+---------------------+------+----+
|Discipline           |Female|Male|
+---------------------+------+----+
|Artistic Swimming    |105   |0   |
|Cycling BMX Freestyle|10    |9   |
|Diving               |72    |71  |
|Rhythmic Gymnastics  |96    |0   |
+---------------------+------+----+



In [0]:
# Display the rolling sum of total women players played in the Olympics
df=df_entries_gender.select("Discipline","Female").withColumn("Rolling_Sum_Female",sum(col("Female")).over(Window.orderBy(col("Female"))))
df.show(truncate=False)

+---------------------+------+------------------+
|Discipline           |Female|Rolling_Sum_Female|
+---------------------+------+------------------+
|Cycling BMX Freestyle|10    |10                |
|Trampoline Gymnastics|16    |26                |
|Sport Climbing       |20    |66                |
|Surfing              |20    |66                |
|Cycling BMX Racing   |24    |90                |
|Marathon Swimming    |25    |115               |
|3x3 Basketball       |32    |147               |
|Modern Pentathlon    |36    |183               |
|Cycling Mountain Bike|38    |221               |
|Karate               |40    |301               |
|Skateboarding        |40    |301               |
|Canoe Slalom         |41    |342               |
|Beach Volleyball     |48    |390               |
|Triathlon            |55    |445               |
|Golf                 |60    |505               |
|Archery              |64    |569               |
|Taekwondo            |65    |634               |


In [0]:
# Display the total count of couches availble for each sport based on country
df_coaches=spark.read.format("csv").option("header",True).option("delimiter",",").option("inferscheam",True).load("/FileStore/tables/Olympic/Coaches.csv")
df=df_coaches.groupBy(col("Discipline")).agg(count(col("Name")).alias("Count_Coaches"))
df.show(truncate=False)

+-----------------+-------------+
|Discipline       |Count_Coaches|
+-----------------+-------------+
|Baseball/Softball|26           |
|Basketball       |74           |
|Handball         |29           |
|Rugby Sevens     |25           |
|Water Polo       |22           |
|Volleyball       |45           |
|Artistic Swimming|69           |
|Hockey           |45           |
|Football         |59           |
+-----------------+-------------+



In [0]:
# display country with the 3 highest count of couches send by them
df=df_coaches.groupBy(col("Country")).agg(count("Name").alias("Total_couches"))
df=df.withColumn("Ranking",dense_rank().over(Window.orderBy("Total_couches"))).filter(col("Ranking")==3).drop("Total_couches")
df.show(truncate=False)

+-----------+-------+
|Country    |Ranking|
+-----------+-------+
|Sweden     |3      |
|Puerto Rico|3      |
|Norway     |3      |
|Ireland    |3      |
|Slovenia   |3      |
|Poland     |3      |
|Austria    |3      |
+-----------+-------+



In [0]:
# Display where number of men players are equal to number of women players and display the sports they played
df=df_entries_gender.filter(col("Female")==col("Male")).select("Discipline")
df.show(truncate=False)

+---------------------+
|Discipline           |
+---------------------+
|3x3 Basketball       |
|Archery              |
|Artistic Gymnastics  |
|Basketball           |
|Beach Volleyball     |
|Canoe Slalom         |
|Cycling BMX Racing   |
|Cycling Mountain Bike|
|Golf                 |
|Handball             |
|Hockey               |
|Marathon Swimming    |
|Modern Pentathlon    |
|Sailing              |
|Shooting             |
|Skateboarding        |
|Sport Climbing       |
|Surfing              |
|Table Tennis         |
|Taekwondo            |
+---------------------+
only showing top 20 rows



In [0]:
# Display the mostly played sport in the event individually by men and by women in seperate row
df_female=df_entries_gender.select("Discipline","Female").withColumn("Ranking_Female",dense_rank().over(Window.orderBy(desc("Female"))))
df_female=df_female.filter(col("Ranking_Female")==1)

df_male=df_entries_gender.select("Discipline","Male").withColumn("Ranking_male",dense_rank().over(Window.orderBy(desc("Male"))))
df_male=df_male.filter(col("Ranking_male")==1)

Final_df=df_male.unionByName(df_female,True).drop("Male","Female")
display(Final_df)

Discipline,Ranking_male,Ranking_Female
Athletics,1,null
Athletics,null,1


In [0]:
# Display the mostly played sport in the event
max_value=df_entries_gender.select(max(col("Total")).alias("Mostly_played")).collect()[0][0]
df=df_entries_gender.filter(col("Total")==max_value).drop("Male","Female")
df.show(truncate=False)

+----------+-----+
|Discipline|Total|
+----------+-----+
|Athletics |2041 |
+----------+-----+



In [0]:
# Display if any country went without any medals
df=df_medals.filter(col("Total")==0)
df.show(truncate=False)

+----+------------+----+------+------+-----+-------------+
|Rank|Team_Country|Gold|Silver|Bronze|Total|Rank_by_Total|
+----+------------+----+------+------+-----+-------------+
+----+------------+----+------+------+-----+-------------+

